In [55]:
import qutip as qt
import numpy as np
import scipy as sp

In [154]:
sx = 0.5*qt.operators.sigmax()
sz = 0.5*qt.operators.sigmaz()
I = qt.operators.identity(2)
sm = qt.operators.sigmam()

In [155]:
def hamiltonian_two_level(N,omega,delta):
  sx = 0.5*qt.operators.sigmax()
  sz = 0.5*qt.operators.sigmaz()
  I = qt.operators.identity(2)

  ham = qt.tensor([I*0]*N)
  for i in range(N):
    hz = qt.tensor([sz if j==i else I for j in range(N)])
    hx = qt.tensor([sx if j==i else I for j in range(N)])
    ham += delta*hz+omega*hx
    
  
  return(ham)

In [156]:
# one spin ising Hamiltonian
N = 1
omega = 1
delta = -1

ham = hamiltonian_two_level(N,omega,delta)
ham

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[-0.5  0.5]
 [ 0.5  0.5]]

In [157]:
# coherent part
I = qt.operators.identity(2)
Id = qt.tensor([I]*N)
coherent = -1j*(qt.tensor(Id,ham)-qt.tensor(ham.trans(),Id))
coherent

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j  0.-0.5j 0.+0.5j 0.+0.j ]
 [0.-0.5j 0.-1.j  0.+0.j  0.+0.5j]
 [0.+0.5j 0.+0.j  0.+1.j  0.-0.5j]
 [0.+0.j  0.+0.5j 0.-0.5j 0.+0.j ]]

In [158]:
# dissipative part
kappa = 1
c_ops = [qt.tensor([sm if i==j  else I for j in range(N)]) for i in range(N)]
c_ops = [np.sqrt(kappa) * c_op for c_op in c_ops]
c_tops = [c.dag() for c in c_ops]
dissipative_part = sum([qt.tensor(Ad.trans(),A)-0.5*qt.tensor(Id,Ad*A)-0.5*qt.tensor((Ad*A).trans(),Id) for Ad,A in zip(c_tops,c_ops)])
dissipative_part

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.  -0.5  0.   0. ]
 [ 0.   0.  -0.5  0. ]
 [ 1.   0.   0.   0. ]]

In [159]:
Linbladian = coherent+dissipative_part
Linbladian

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-1. +0.j   0. -0.5j  0. +0.5j  0. +0.j ]
 [ 0. -0.5j -0.5-1.j   0. +0.j   0. +0.5j]
 [ 0. +0.5j  0. +0.j  -0.5+1.j   0. -0.5j]
 [ 1. +0.j   0. +0.5j  0. -0.5j  0. +0.j ]]

In [160]:
eigvals, eigmat = Linbladian.eigenstates()

In [ ]:
idx = np.argsort(abs(eigvals))[0] # minimum eigen value is zero which corresponds to steady state solutions
print(idx)

rho_00 = eigmat[idx].full().reshape(2**N,2**N)
print(rho_00)

3
[[0.14586499+0.j         0.29172998-0.14586499j]
 [0.29172998+0.14586499j 0.87518995+0.j        ]]


In [180]:
# checking the eigen value equation

Linbladian*rho_00.reshape(2**N*2**N,1) # zero vector

array([[-1.38777878e-16-2.77555756e-17j],
       [-3.05311332e-16+5.55111512e-17j],
       [ 5.55111512e-17+5.55111512e-17j],
       [ 1.38777878e-16+2.77555756e-17j]])

In [ ]:
# checking the steady state solution
qt.steadystate(ham,c_ops) # why the off-diagonal elements are interchanged? - the result is the transposed version of what i got

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.14285714+0.j         0.28571429-0.14285714j]
 [0.28571429+0.14285714j 0.85714286+0.j        ]]

# EVD using Numpy

In [172]:
val,mat = np.linalg.eig(Linbladian)

In [176]:
indx = np.argsort(abs(val))[0]
print(indx)

print(mat[indx].reshape(2**N,2**N))

2
[[-0.14092519+0.03636657j  0.85841332+0.j        ]
 [ 0.29172998-0.14586499j -0.47627547-0.12290568j]]


The matrix is not physical, the trace of the matrix is not equal to one and matrix is not hermitian

In [ ]:
# enforcing Hermicity and trace preservation

m = mat[indx].reshape(2**N,2**N)
rho = (m+(m.conj()).T)/2
rho_0 = rho/np.trace(rho)
rho_0

array([[ 0.22832962-0.j        , -0.9317418 -0.11816659j],
       [-0.9317418 +0.11816659j,  0.77167038-0.j        ]])

Not same as the steady state we got

In [178]:
# Checking the eigen value equation

print(Linbladian*rho_0.reshape(4,1))

[[-0.34649621+0.j        ]
 [ 0.34770431+1.26249547j]
 [ 0.34770431-1.26249547j]
 [ 0.34649621+0.j        ]]


The reseult should be a zero vector ideally

In [181]:
val[indx]*rho_0.reshape(4,1)

array([[ 6.23648376e-17+3.37756207e-18j],
       [-2.52743426e-16-4.60582192e-17j],
       [-2.56239381e-16+1.84926696e-17j],
       [ 2.10770278e-16+1.14149210e-17j]])

## Checking diagonalization using Scipy

In [182]:
import scipy as sp

In [183]:
sp.sparse.linalg.eigsh(Linbladian.full(),k=2, which='SM') # works only with Hermitian matrix

(array([ 2.39414570e-17, -7.58055872e-01]),
 array([[-0.04927787-0.13728906j,  0.17136662-0.47821476j],
        [ 0.03873333-0.323856j  , -0.04496608+0.48981852j],
        [-0.2358448 -0.22530026j, -0.27636858+0.40689622j],
        [-0.29566722-0.82373439j, -0.17136662+0.47821476j]]))

In [184]:
sp.linalg.eig(Linbladian.full())

(array([-6.20972064e-01-1.38669775e+00j, -6.20972064e-01+1.38669775e+00j,
         2.73135115e-16+1.47924831e-17j, -7.58055872e-01-1.86951828e-16j]),
 array([[ 0.3319465 -1.03844031e-01j,  0.3319465 +1.03844031e-01j,
          0.14586499+2.18575158e-16j,  0.507992  +0.00000000e+00j],
        [ 0.85841332+0.00000000e+00j, -0.14092519-3.63665738e-02j,
          0.29172998+1.45864991e-01j, -0.47627547+1.22905681e-01j],
        [-0.14092519+3.63665738e-02j,  0.85841332+0.00000000e+00j,
          0.29172998-1.45864991e-01j, -0.47627547-1.22905681e-01j],
        [-0.3319465 +1.03844031e-01j, -0.3319465 -1.03844031e-01j,
          0.87518995+0.00000000e+00j, -0.507992  +1.38777878e-16j]]))

same problem as Numpy

### Checking the Diagonalization of Linbladian

In [196]:
import sympy as sp

In [197]:
e = sp.symbols('\epsilon')
d = sp.symbols('\Delta')
g = sp.symbols('\gamma')

In [198]:
# Hardcoding Linbladian

L = sp.Matrix([[-g,-1j*e/2,1j*e/2,0],
               [-1j*e/2, -0.5*g+1j*d,0,1j*e/2],
               [1j*e/2,0,-0.5*g-1j*d,-1j*e/2],

               [g,1j*e/2,-1j*e/2,0]])

In [199]:
vals = L.eigenvals()
vals

{-0.666666666666667*\gamma - 0.139991227766097*(-3.0*\Delta**2 - 3.0*\epsilon**2 + 0.25*\gamma**2)/(\Delta**2*\gamma + 0.5*\epsilon**2*\gamma + 0.842592592592593*\gamma**3 - 0.666666666666667*\gamma*(1.0*\Delta**2 + 1.0*\epsilon**2 + 1.25*\gamma**2) + sqrt(-0.148148148148148*(-\Delta**2 - \epsilon**2 + 0.0833333333333333*\gamma**2)**3 + (\Delta**2*\gamma + 0.5*\epsilon**2*\gamma + 0.842592592592593*\gamma**3 - 0.666666666666667*\gamma*(1.0*\Delta**2 + 1.0*\epsilon**2 + 1.25*\gamma**2))**2))**(1/3) - 0.7937005259841*(\Delta**2*\gamma + 0.5*\epsilon**2*\gamma + 0.842592592592593*\gamma**3 - 0.666666666666667*\gamma*(1.0*\Delta**2 + 1.0*\epsilon**2 + 1.25*\gamma**2) + sqrt(-0.148148148148148*(-\Delta**2 - \epsilon**2 + 0.0833333333333333*\gamma**2)**3 + (\Delta**2*\gamma + 0.5*\epsilon**2*\gamma + 0.842592592592593*\gamma**3 - 0.666666666666667*\gamma*(1.0*\Delta**2 + 1.0*\epsilon**2 + 1.25*\gamma**2))**2))**(1/3): 1,
 -0.666666666666667*\gamma - 0.139991227766097*(-3.0*\Delta**2 - 3.0*\e

In [200]:
eig_vals = [keys.subs({d:-1,g:1,e:1}) for keys in vals.keys()] # substiuting values

In [201]:
eig_mat = L.eigenvects()

In [202]:
eig_mat


[(0,
  1,
  [Matrix([
   [                               \epsilon**2/(4.0*\Delta**2 + \epsilon**2 + \gamma**2)],
   [(-2.0*\Delta*\epsilon + I*\epsilon*\gamma)/(4.0*\Delta**2 + \epsilon**2 + \gamma**2)],
   [(-2.0*\Delta*\epsilon - I*\epsilon*\gamma)/(4.0*\Delta**2 + \epsilon**2 + \gamma**2)],
   [                                                                                 1.0]])]),
 (-0.666666666666667*\gamma - 0.139991227766097*(-3.0*\Delta**2 - 3.0*\epsilon**2 + 0.25*\gamma**2)/(\Delta**2*\gamma + 0.5*\epsilon**2*\gamma + 0.842592592592593*\gamma**3 - 0.666666666666667*\gamma*(\Delta**2 + \epsilon**2 + 1.25*\gamma**2) + (-0.148148148148148*(-\Delta**2 - \epsilon**2 + 0.0833333333333333*\gamma**2)**3 + (\Delta**2*\gamma + 0.5*\epsilon**2*\gamma + 0.842592592592593*\gamma**3 - 0.666666666666667*\gamma*(\Delta**2 + \epsilon**2 + 1.25*\gamma**2))**2)**0.5)**0.333333333333333 - 0.7937005259841*(\Delta**2*\gamma + 0.5*\epsilon**2*\gamma + 0.842592592592593*\gamma**3 - 0.66666666666666

In [203]:
print('rho_ss',eig_mat[0][2][0].subs({d:-1,g:1,e:1}).reshape(2,2))

rho_ss Matrix([[0.166666666666667, 0.333333333333333 + 0.166666666666667*I], [0.333333333333333 - 0.166666666666667*I, 1.00000000000000]])


# Conclusion

- EigenMatrices given by np.linalg.eig and scipy.eig is not correct 
- validated the consistency of qutip results with quatumoptics.jl package